In [43]:
import os
import sys
import json
import pathlib
import numpy as np

sys.path.append("/home/arnaik/OracleProject")

from src.datautils import read_jsonl

In [58]:
def load_linter_results(text):
    results = []
    idiom_code = None
    for line in text.split("\n"):
        line = line.strip()
        if line == "": continue
        elif line.startswith("**Idiom") and line.endswith("Violations:**"):
            idiom_code = line.removesuffix("Violations:**").removeprefix("**Idiom").strip()
        elif line == "NO VIOLATIONS FOUND": continue
        else:
            try: 
                result = json.loads(line)
                result["code"] = idiom_code
                results.append(result)
            except Exception as e: pass
                # print(e)
                # print(f"{e}: {line}")
    return results

In [3]:
cd /home/arnaik/OracleProject

/home/arnaik/OracleProject


In [23]:
for step in [0, 200, 400, 600, 800, 1000, 1200, 1240]:
    if step == 0:
        preds = read_jsonl("data/meta_linting_preds_vllm/qwen3_4b_sft_preds_4000_transfer_v5_lineno.jsonl")
    else: preds = read_jsonl(f"data/meta_linting_preds_vllm/qwen3_4b_dpo_preds_{step}_transfer_v5_lineno.jsonl")
    no_violation_acc = 0
    no_violation_tot = 0
    for rec in preds:
        # print(rec['ground_truth'])
        gt_results = load_linter_results(rec['ground_truth'])
        if len(gt_results) == 0:
            no_violation_tot += 1
            model_results = load_linter_results(rec['model_response'])
            if len(model_results) == 0:
                no_violation_acc += 1

    print(f"DPO {step} no-violation pred acc (%): {100*no_violation_acc/no_violation_tot:.2f}")

5300it [00:00, 52081.18it/s]


DPO 0 no-violation pred acc (%): 99.13


5300it [00:00, 328420.69it/s]


DPO 200 no-violation pred acc (%): 79.35


5300it [00:00, 226324.42it/s]


...

DPO 1000 no-violation pred acc (%): 75.81


5300it [00:00, 325506.44it/s]


DPO 1200 no-violation pred acc (%): 75.81


5300it [00:00, 327766.97it/s]


DPO 1240 no-violation pred acc (%): 75.45


In [4]:
import json

dpo_self_samples = read_jsonl("data/dpo_self_samples/qwen3_4b_think_mode_untrained.jsonl")

0it [00:00, ?it/s]

53000it [00:02, 17992.88it/s]


In [38]:
sft_data = {rec['id']: rec for rec in json.load(open("data/ruff_meta_linting/train_v5.json"))}

In [39]:
print(len(sft_data))
print(sft_data['UP005_bf3144d239c15b63bb40f7f245d1d9ea994d758f'])

53000
{'id': 'UP005_bf3144d239c15b63bb40f7f245d1d9ea994d758f', 'messages': [{'content': 'Look at the following list of code idiom specifications with definitions and examples:\n# Idiom UP005 (deprecated-unittest-alias)\n\nDefinition: Checks for uses of deprecated methods from the unittest module.\n\nRationale: The unittest module has deprecated aliases for some of its methods.\nThe deprecated aliases were removed in Python 3.12. Instead of aliases,\nuse their non-deprecated counterparts.\n\nExample:\nfrom unittest import TestCase\n\n\nclass SomeTest(TestCase):\n    def test_something(self):\n        self.assertEquals(1, 1)\n\n\n\nUse instead:\n\n\nfrom unittest import TestCase\n\n\nclass SomeTest(TestCase):\n    def test_something(self):\n        self.assertEqual(1, 1)\n\nGiven these idioms, your task is to look at a code file and detect violations of the above idioms, and flag them like a linter. You should also suggest a fix if possible. Report the results per idiom specification men

In [73]:
dpo_self_samples = read_jsonl("data/dpo_self_samples/qwen3_4b_think_mode_untrained.jsonl")
sft_data = {rec['id']: rec for rec in json.load(open("data/ruff_meta_linting/train_v5.json"))}
top_k = 2

high_reward_concise_sft_data = []
for sample in dpo_self_samples:
    sft_instance = sft_data[sample['id']]
    valid_responses = []
    for model_response in sample["model_responses"]:
        reward = model_response[1]
        response = model_response[0]
        if reward == 1 and "</think>" in response and "### Final Idiom Violations Found" in response and "**Idiom " in response and "Violations:**" in response:
            valid_responses.append(response)
    if len(valid_responses) > 0:
        # index = np.argmin([len(response) for response in valid_responses])
        if len(load_linter_results(sample['ground_truth'])) == 0: K = 1 # allow only one response for NO VIOLATION cases.
        else: K = min(top_k, len(valid_responses)) # allow up to 2 responses for >= 1 VIOLATION cases.
        indices = np.argpartition([len(response) for response in valid_responses], K-1)[:K]
        for index in indices:
            selected_response = valid_responses[index]
            sft_instance['messages'][1]['content'] = selected_response
            high_reward_concise_sft_data.append(sft_instance)

print(high_reward_concise_sft_data[0])
print(len(high_reward_concise_sft_data))

53000it [00:02, 21419.46it/s]


{'id': 'UP005_bf3144d239c15b63bb40f7f245d1d9ea994d758f', 'messages': [{'content': 'Look at the following list of code idiom specifications with definitions and examples:\n# Idiom UP005 (deprecated-unittest-alias)\n\nDefinition: Checks for uses of deprecated methods from the unittest module.\n\nRationale: The unittest module has deprecated aliases for some of its methods.\nThe deprecated aliases were removed in Python 3.12. Instead of aliases,\nuse their non-deprecated counterparts.\n\nExample:\nfrom unittest import TestCase\n\n\nclass SomeTest(TestCase):\n    def test_something(self):\n        self.assertEquals(1, 1)\n\n\n\nUse instead:\n\n\nfrom unittest import TestCase\n\n\nclass SomeTest(TestCase):\n    def test_something(self):\n        self.assertEqual(1, 1)\n\nGiven these idioms, your task is to look at a code file and detect violations of the above idioms, and flag them like a linter. You should also suggest a fix if possible. Report the results per idiom specification mentioned

In [74]:
from collections import defaultdict

sft_data = json.load(open("data/ruff_meta_linting/train_v5.json"))

violation_dist = defaultdict(lambda: 0)
for index in range(len(high_reward_concise_sft_data)):
    response = high_reward_concise_sft_data[index]['messages'][1]['content']
    # print(response)
    violation_dist[len(load_linter_results(response))] += 1

print("CoT SFT data:")
violation_dist = dict(violation_dist)
print(violation_dist)
print("NO VIOLATIONS:", violation_dist[0])
print(">=1 VIOLATIONS:", sum(v for k,v in violation_dist.items() if k != 0))

violation_dist = defaultdict(lambda: 0)
for index in range(len(sft_data)):
    response = sft_data[index]['messages'][1]['content']
    # print(response)
    violation_dist[len(load_linter_results(response))] += 1

print("Ruff SFT data:")
violation_dist = dict(violation_dist)
print(violation_dist)
print("NO VIOLATIONS:", violation_dist[0])
print(">=1 VIOLATIONS:", sum(v for k,v in violation_dist.items() if k != 0))

CoT SFT data:
{0: 37191, 1: 12139, 2: 2017, 4: 337, 6: 144, 3: 554, 9: 17, 10: 16, 5: 144, 14: 6, 7: 29, 8: 59, 12: 10, 13: 1, 11: 6, 22: 2, 15: 1, 16: 2}
NO VIOLATIONS: 37191
>=1 VIOLATIONS: 15484
Ruff SFT data:
{0: 37534, 1: 10063, 2: 2312, 4: 610, 3: 919, 5: 348, 89: 1, 8: 133, 6: 229, 12: 71, 9: 98, 7: 185, 22: 11, 16: 29, 14: 52, 15: 25, 10: 85, 13: 34, 29: 6, 11: 61, 44: 3, 25: 5, 35: 5, 27: 9, 26: 5, 20: 21, 50: 3, 31: 5, 18: 16, 17: 17, 190: 1, 36: 2, 23: 8, 19: 18, 30: 2, 28: 7, 21: 18, 212: 1, 52: 1, 40: 3, 37: 1, 41: 2, 33: 8, 46: 1, 54: 2, 42: 2, 49: 1, 139: 1, 76: 1, 88: 1, 45: 1, 38: 3, 64: 1, 32: 1, 102: 1, 24: 6, 34: 1, 107: 1, 53: 1, 75: 1, 72: 1, 51: 2, 77: 1, 104: 2, 59: 1}
NO VIOLATIONS: 37534
>=1 VIOLATIONS: 15466


In [ ]:
# 53000 -> 45679 if each input is only allowed once
# 5300 ->  if up to 2 responses can be kept per input (we chose the shortest 2 CoTs) but only for cases other than "NO VIOLATIONS" (at least 1 violation cases).

In [76]:
with open("data/ruff_meta_linting/train_v5_cot.json", "w") as f:
    json.dump(high_reward_concise_sft_data, f, indent=4)